In [6]:
import tushare as ts
import numpy as np
import json
import datetime
import time
import download_financial_data_report as dr
#发现均线重合股票并查找财务数据
# init environment
ts.set_token('464a66ad8b85aeba451b3703dad3e844cc0fcc6dba43321fe8de41da')
pro = ts.pro_api()

resultList = []

# analyse methods
names1 = ['报表日期', '资产总计', '负债合计', '所有者权益(或股东权益)合计', '货币资金', '短期借款', '应收票据及应收账款', '应收票据', '应收账款', '应付票据', '应付账款',
          '预收款项', '存货', '流动资产合计', '在建工程(合计)', '无形资产', '商誉', ' ', '应付职工薪酬', '其他应付款', '其他流动负债', '股东权益合计',
          '负债和所有者权益(或股东权益)总计', '负债及股东权益总计','所有者权益合计']
names2 = ['报表日期', '一、营业收入', '营业收入', '营业成本', '一、营业总收入','二、营业支出', '三、营业利润', '五、净利润', '基本每股收益(元/股)', '销售费用', '管理费用', '财务费用', '研发费用', '少数股东损益',
          '归属于母公司所有者的综合收益总额']
names3 = ['报表日期', '经营活动现金流入小计', '投资活动现金流入小计', '筹资活动现金流入小计', '资产减值准备', '固定资产折旧、油气资产折耗、生产性物资折旧', '无形资产摊销',
          '现金及现金等价物的净增加额']
rule_rough_net=0.4
rule_roe=0.1
rule_property=1000000000
def analyzeStock(row,start_date,end_date,count):
    # print("IN ANALYSE...WITH CODE", str(row[0]))
    time.sleep(0.15)
    df = ts.pro_bar(ts_code=str(row[0]), start_date=str(start_date), end_date=str(end_date), ma=[5, 10, 20, 30, 60])
    # print('df')
    # print(df)
    if df.empty == False:
        # print(df.iloc[0])
        close = df.iloc[0, 5]
        ma5 = df.iloc[0, 11]
        ma10 = df.iloc[0, 13]
        ma20 = df.iloc[0, 15]
        ma30 = df.iloc[0, 17]
        ma60 = df.iloc[0, 19]
        numList = [ma5, ma10, ma20, ma30, ma60, close]

        maxNum = max(numList)
        minNum = min(numList)

        row = np.append(row, (maxNum - minNum) / maxNum)

        if (maxNum-minNum) < 0.02*maxNum:
            print("*************BINGO!*************", count)
            pure_income = dr.dataMatrix(row[1], names1, names2, names3, 'year', count)
            pure_income['type'] = ['净利润', '所有者权益', 'ROE', '毛利率', '总资产', '负债率']

            if (pure_income is not None):
                len1 = pure_income.shape[1]  # 返回列数
                if len1 > 2:  # 上市时间大于两年
                    if pure_income.iloc[2, 0] > rule_roe and pure_income.iloc[2, 1] > rule_roe and pure_income.iloc[
                        2, 2] > rule_roe \
                            and pure_income.iloc[3, 0] > rule_rough_net and pure_income.iloc[
                        3, 1] > rule_rough_net and pure_income.iloc[
                        3, 2] > rule_rough_net \
                            and pure_income.iloc[4, 0] > rule_property:
                        print("pure_income:\n{0}".format(pure_income))
                        print("   ---   ")
                        print(str(count))
                        print("   --连续三年毛利率大于:   ")
                        print(str(rule_rough_net))
                        print("   --连续三年ROE大于:   ")
                        print(str(rule_roe))
                        print("   --总资产大于:   ")
                        print(str(rule_property))
                        resultList.append(pure_income)

                elif len1 > 1 and len1 <= 2:  # 上市时间满两年
                    print(stocks.values)
                    if pure_income.iloc[2, 0] > rule_roe and pure_income.iloc[2, 1] > rule_roe and pure_income.iloc[
                        3, 0] > rule_rough_net \
                            and pure_income.iloc[3, 1] > rule_rough_net and pure_income.iloc[4, 0] > rule_property:
                        print("pure_income:\n{0}".format(pure_income))
                        print("   ---   ")
                        print(str(count))
                        print("   --连续两年毛利率大于:   ")
                        print(str(rule_rough_net))
                        print("   --连续两年ROE大于:   ")
                        print(str(rule_roe))
                        print("   --总资产大于:   ")
                        print(str(rule_property))
                        resultList.append(pure_income)


            print(numList)
            print(row)
            resultList.append(row)

            return 0

        return 1

        # print(resultList)


# list all the stocks
stocks = pro.stock_basic(exchange='', list_status='L', fields='ts_code,symbol,name,area,industry,list_date')

# stocks = pro.hk_basic()

# calculate average value one by one
count = 0

print("\n\n\nSTART..............FIND STOCKS..........\n")
for row in stocks.values:

    # print("\n\n\nSTART..............ANALYSE\n")
    today = time.strftime("%Y%m%d", time.localtime(time.time()))
    # today = time.strftime('%Y%m%d',time.localtime(time.time()))
    start = (datetime.date.today() - datetime.timedelta(days=120)).strftime("%Y%m%d")
    # print(today)
    # print(start)
    # print(row)
    if analyzeStock(row, start, today,count) == 0:
        count = count + 1

    # if i > 1000:
    #    break

file = open("files/ave_value.txt", "a")
file.write("******************************************")
file.write(time.strftime("%Y%m%d", time.localtime(time.time())))
file.write("******************************************\n")

for val in resultList:

    file.write(str(val))
    file.write("\n")
    # print(str(val))
file.write("\n\n\n")
file.close()
print(resultList)







START..............FIND STOCKS..........

*************BINGO!************* 0
code init: 000150
000150_宜华健康
code: 000150 path_file: /Users/yuqing/PycharmProjects/stocks/files/finance/2019/000150_宜华健康
[3.724, 3.77, 3.7485, 3.777, 3.7805, 3.72]
['000150.SZ' '000150' '宜华健康' '广东' '医疗保健' '20000807' 0.016003174183309026]
*************BINGO!************* 1
code init: 000409
000409_ST地矿
code: 000409 path_file: /Users/yuqing/PycharmProjects/stocks/files/finance/2019/000409_ST地矿
[4.186, 4.204, 4.212, 4.2563, 4.1992, 4.18]
['000409.SZ' '000409' 'ST地矿' '山东' '综合类' '19960627' 0.017926367972182576]
*************BINGO!************* 2
code init: 000422
000422_ST宜化
code: 000422 path_file: /Users/yuqing/PycharmProjects/stocks/files/finance/2019/000422_ST宜化
[2.586, 2.579, 2.547, 2.5597, 2.5948, 2.59]
['000422.SZ' '000422' 'ST宜化' '湖北' '农药化肥' '19960815' 0.018421458301217844]
*************BINGO!************* 3
code init: 000553
000553_安道麦A
code: 000553 path_file: /Users/yuqing/PycharmProjects/stocks/files/

KeyboardInterrupt: 

In [ ]:
#查找每一只股票的财务数据

In [9]:
import pandas as pd
from flask import Flask, request, jsonify
import os
import matplotlib.pyplot as plt
import tushare as ts
import numpy as np
import datetime
import time, json

#查找每一只股票的财务数据
# init environment
ts.set_token('464a66ad8b85aeba451b3703dad3e844cc0fcc6dba43321fe8de41da')
pro = ts.pro_api()
np.set_printoptions(threshold = np.inf)
#若想不以科学计数显示:
np.set_printoptions(suppress = True)
#显示所有列
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows',None)

rule_rough_net=0.35
rule_roe=0.15
rule_property=5000000000





def read(code, names1, names2, names3, period, count):
    datalist = []
    filepath = '/Users/yuqing/PycharmProjects/stocks/files/finance/2019/'

    dirlist = os.listdir(filepath)

    print("code init:", code)
    path_file = ''
    for dirname in dirlist:
        if code in dirname:
            print(dirname)
            path_file = filepath + dirname
            break;
        else:
            continue;
    print("code:", code, "path_file:",path_file)
    if code in path_file:

        df1 = pd.read_csv(path_file + '/bs.xls', sep='\t', encoding="GBK")  # vDOWN_BalanceSheet
        df2 = pd.read_csv(path_file + '/ps.xls', sep='\t', encoding="GBK")  # vDOWN_ProfitStatement
        df3 = pd.read_csv(path_file + '/cf.xls', sep='\t', encoding="GBK")  # vDOWN_CashFlow

        # 这个会直接默认读取到这个Excel的第一个表单
        if period == 'year':
            columns1 = df1.columns.values.tolist()
            years1 = list(filter(lambda x: x.endswith('1231') and int(x) >= 20101231, columns1))
            years = years1
            columns2 = df2.columns.values.tolist()
            years2 = list(filter(lambda x: x.endswith('1231') and int(x) >= 20101231, columns2))
            if len(years2) > len(years):
                years = years2
            columns3 = df3.columns.values.tolist()
            years3 = list(filter(lambda x: x.endswith('1231') and int(x) >= 20101231, columns3))
            if len(years3) > len(years):
                years = years3
            years1.append('报表日期')
            years2.append('报表日期')
            years3.append('报表日期')
            data0 = df1.loc[(df1['报表日期'].isin(names1))][years1]
            data1 = df2.loc[(df2['报表日期'].isin(names2))][years2]
            data2 = df3.loc[(df3['报表日期'].isin(names3))][years3]
        else:
            data0 = df1.loc[(df1['报表日期'].isin(names1))]
            data1 = df2.loc[(df2['报表日期'].isin(names2))]
            data2 = df3.loc[(df3['报表日期'].isin(names3))]

        # 默认读取前5行的数据
        datalist.append(data0)
        datalist.append(data1)
        datalist.append(data2)
        datalist.append(years)
        #


        # data0.append(data1, ignore_index=True)
        # data0.append(data2, ignore_index=True)

        # print("data0获取到所有的值:\n{0}".format(data0.values))

    else:
        print(code, " data not found! ")


    return datalist
    # 格式化输出


def dataMatrix(code,names1,names2,names3,way,count):

    datalist = read(code, names1, names2, names3, way, count)

    if len(datalist) > 0:

        # print("datalist获取到所有的值:\n{0}".format(datalist))

        df1 = datalist[0]
        df2 = datalist[1]
        df3 = datalist[2]

        year = datalist[3]

        # print("len:", len1)
        # year = year[0:len-1]

        # 计算ROE 2
        pure_income = pd.DataFrame(df2[(df2['报表日期'].isin(['五、净利润']))], columns=year)
        # print("pure_income:\n", pure_income)
        owner_rights = pd.DataFrame(df1[(df1['报表日期'].isin(['所有者权益(或股东权益)合计', '股东权益合计', '所有者权益合计']))], columns=year)
        # pure_income.iloc['净资产收益率'] = np.log(pure_income.iloc[0] / owner_rights.iloc[0])
        pure_income1 = pure_income.append(owner_rights).drop(['报表日期'], axis=1).reset_index(drop=True).astype(float)
        pure_income1.loc[2] = pure_income1.loc[0] / pure_income1.loc[1].values
        # print("pure_income:\n", pure_income1)

        # 计算毛利率  3

        income_cost = pd.DataFrame(df2[(df2['报表日期'].isin(['营业成本', '二、营业支出']))], columns=year)
        # print("pure_income:\n", pure_income)
        income = pd.DataFrame(df2[(df2['报表日期'].isin(['一、营业收入', '营业收入']))], columns=year)
        # pure_income.iloc['净资产收益率'] = np.log(pure_income.iloc[0] / owner_rights.iloc[0])
        income_cost = income_cost.append(income).drop(['报表日期'], axis=1).reset_index(drop=True).astype(float)
        pure_income1.loc[3] = (income_cost.loc[1] - income_cost.loc[0]) / income_cost.loc[1].values

        # 总资产 > 100亿
        total_assets = pd.DataFrame(df1[(df1['报表日期'].isin(['负债和所有者权益(或股东权益)总计', '负债及股东权益总计']))], columns=year)
        total_assets = total_assets.drop(['报表日期'], axis=1).reset_index(drop=True).astype(float)
        pure_income1 = pure_income1.append(total_assets).reset_index(drop=True)

        # 计算负债率
        debt = pd.DataFrame(df1[(df1['报表日期'].isin(['负债合计']))], columns=year).drop(['报表日期'], axis=1).reset_index(
            drop=True).astype(float)
        pure_income1.loc[5] = debt.loc[0] / total_assets.loc[0].values
        return pure_income1

    else:
        print("no data for this code")
        return None


def dfToJson(row, matrix,count):
    df_json = matrix.to_json(orient='index', force_ascii=False)
    jsn = {'index':count,'all_code':row[0], 'code':row[1], 'name':row[2], 'location':row[3], 'type':row[4], 'show_date':row[5], 'value':df_json}
    return jsn

def writeToFile(count,jsn):
    file = open("files/fnl_report.txt", "a")
    file.write("\n")
    file.write(time.strftime("%Y%m%d", time.localtime(time.time())))
    file.write("   ---   ")
    file.write(str(count))
    file.write("   --连续三年毛利率大于:   ")
    file.write(str(rule_rough_net))
    file.write("   --连续三年ROE大于:   ")
    file.write(str(rule_roe))
    file.write("   --总资产大于:   ")
    file.write(str(rule_property))
    file.write("\n")
    file.write(str(jsn))
    file.write("\n")


# list all the stocks
stocks = pro.stock_basic(exchange='', list_status='L', fields='ts_code,symbol,name,area,industry,list_date')

count = 0

names1 = ['报表日期', '资产总计', '负债合计', '所有者权益(或股东权益)合计', '货币资金', '短期借款', '应收票据及应收账款', '应收票据', '应收账款', '应付票据', '应付账款',
          '预收款项', '存货', '流动资产合计', '在建工程(合计)', '无形资产', '商誉', ' ', '应付职工薪酬', '其他应付款', '其他流动负债', '股东权益合计',
          '负债和所有者权益(或股东权益)总计', '负债及股东权益总计','所有者权益合计']
names2 = ['报表日期', '一、营业收入', '营业收入', '营业成本', '一、营业总收入','二、营业支出', '三、营业利润', '五、净利润', '基本每股收益(元/股)', '销售费用', '管理费用', '财务费用', '研发费用', '少数股东损益',
          '归属于母公司所有者的综合收益总额']
names3 = ['报表日期', '经营活动现金流入小计', '投资活动现金流入小计', '筹资活动现金流入小计', '资产减值准备', '固定资产折旧、油气资产折耗、生产性物资折旧', '无形资产摊销',
          '现金及现金等价物的净增加额']
result = []


def all_stocks():
    for row in stocks.values:
        ts_code = str(row[0])
        code = ts_code[0:6]
        print("\n\n\nSTART..............ANALYSE\t", code, "\n\n")
        pure_income1 = dataMatrix(code, names1, names2, names3, 'year', count)
        if(pure_income1 is not None):
            len1 = pure_income1.shape[1] #返回列数
            if len1 > 2:  # 上市时间大于两年
                if pure_income1.iloc[2, 0] > rule_roe and pure_income1.iloc[2, 1] > rule_roe and pure_income1.iloc[2, 2] > rule_roe \
                        and pure_income1.iloc[3, 0] > rule_rough_net and pure_income1.iloc[3, 1] > rule_rough_net and pure_income1.iloc[
                    3, 2] > rule_rough_net \
                        and pure_income1.iloc[4, 0] > rule_property:
                    count = count + 1
                    pure_income1['type'] = ['净利润', '所有者权益', 'ROE', '毛利率', '总资产', '负债率']
                    # print("pure_income1.iloc[2,0]",pure_income1.iloc[2,0])
                    # print("pure_income1.iloc[3,0]",pure_income1.iloc[3,0])
                    jsn = dfToJson(row, pure_income1,count)
                    result.append(jsn)


            elif len1 > 1 and len1 <= 2:  # 上市时间满两年
                print(stocks.values)
                if pure_income1.iloc[2, 0] > rule_roe and pure_income1.iloc[2, 1] > rule_roe and pure_income1.iloc[3, 0] > rule_rough_net \
                        and pure_income1.iloc[3, 1] > rule_rough_net and pure_income1.iloc[4, 0] > rule_property:
                    count = count + 1
                    pure_income1['type'] = ['净利润', '所有者权益', 'ROE', '毛利率', '总资产', '负债率']
                    # print("pure_income1.iloc[2,0]",pure_income1.iloc[2,0])
                    # print("pure_income1.iloc[3,0]",pure_income1.iloc[3,0])
                    jsn = dfToJson(row, pure_income1,count)
                    result.append(jsn)
    print("json得出所有的值:\n{0}".format(result))
    writeToFile(count, result)



In [11]:
#5天4次，降低5%的成本价的计划
#input ，crt_price, crt_count, charge_percent 0.03%

import pandas as pd
import requests
import tushare as ts
import numpy as np
import datetime
import time, json

ts.set_token('464a66ad8b85aeba451b3703dad3e844cc0fcc6dba43321fe8de41da')
pro = ts.pro_api()
np.set_printoptions(threshold = np.inf)
#若想不以科学计数显示:
np.set_printoptions(suppress = True)
#显示所有列
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows',None)

#计算上一个月每天可能的成本降低百分比
def posibility_in_a_day(code, asset):
    today = time.strftime("%Y%m%d", time.localtime(time.time()))
    # today = time.strftime('%Y%m%d',time.localtime(time.time()))
    end = (datetime.date.today() - datetime.timedelta(days=14)).strftime("%Y%m%d")
    df = ts.pro_bar(ts_code=str(code), adj='qfq', start_date=str(end), end_date=str(today), asset=str(asset))
    # print(code, df)
    gap = pd.to_numeric(df['high']) - pd.to_numeric(df['low'])

    result = gap.sum()/len(gap)
    # print(code,gap)
    # print((gap.sum()-gap.min()-gap.max())/(len(gap)-2)*0.8*5)
    # print(gap.mean(), gap.min(), gap.max(), gap.std())

    return result

#price 持仓价格
#count 持仓数量
#percent
#code 股票代码
#money 现金


#返回当前单价
def sell(price, count, aim_price, aim_sell):
    left_count = round(count - aim_sell,1)
    sell_money = round(aim_price * aim_sell,3)
    total_money = round(price*count - sell_money*(1-0.00122), 3)
    earn = round(sell_money*(1-0.00122), 3)
    crt_price = round(total_money/left_count,3)
    profit = (aim_price-price)*aim_sell
    print("卖出价格：", aim_price,"卖出数量：", aim_sell,"回收资金：", sell_money, "资本增加：", earn, "盈利：", profit, "单价变为：", crt_price)
    file.write("\n")
    file.write("卖出价格："+str(aim_price)+" 卖出数量："+str(aim_sell)+" 回收资金："+str(sell_money)+ " 资本增加："+str( earn) +" 盈利："+str(profit)+" 单价变为："+str(crt_price))
    return [crt_price, left_count, earn] #成本单价，股票总数，盈利

#返回当前单价
def buy(price,count,aim_price,add_money):
    aim_count = round(add_money/aim_price/100,0)*100 #用增加的钱能买多少
    total_money = round(aim_price*aim_count * (1 + 0.00002)+count*price, 3) #成本总共多少钱
    cost = round(total_money/(aim_count+count), 3) #成本单价
    lower = round(price-cost, 3) #成本单价降低了多少
    print("买入价格：", aim_price, "买入数量：", aim_count, "使用资金：", add_money, "单价变为：", cost, "降低：", lower)
    file.write("\n")
    file.write("买入价格："+ str(aim_price)+ "买入数量："+ str(aim_count)+ "使用资金："+ str(add_money)+"单价变为："+ str(cost)+ "降低："+ str(lower))
    return [cost, aim_count+count]  #成本单价, 股票总数

# def lower_buy(price,count,aim_price,moneylist):
#     money = moneylist[0] #原成本
#     add = moneylist[1]   #增加成本
#     aim_count = round(add/aim_price/100,0)*100 #用所有包括增加的钱能买多少
#     more = round(add * 0.00002, 3) #手续费
#     total_money = round(add+money+more, 3) #成本总共多少钱
#
#     cost = [round(total_money/(aim_count+count), 3), (aim_count+count)] #成本单价, 买入之后的股票总数
#     lower = round(price-cost[0], 3) #成本单价降低了多少
#     print("买入价格：", aim_price, "买入数量：", aim_count, "总数量：", aim_count+count, "使用资金：", add, "手续费：", more, "成本单价：", cost[0],"降低：",lower)
#     return cost


def lower(price, count, code, sinacode, moneyadd, type):
    #根据昨天差价计算今天可能赚到的差价
    gap = posibility_in_a_day(code, type)/price
    #downto = round((gap.sum()-gap.min()-gap.max())/(len(gap)-2)*0.6,3)
    #gap = 0.016 #5%的网格
    sell_percent = 0.9
    downto = gap / 2


    #卖出，印花税0.001, 过户费0.00002, 手续费0.0002
    #买入，手续费0.0002
    #目标是两周之内降低成本价的计划
    #print("\n\n",code,"目标：可降低成本价", round(downto*5, 3), '至', round(price-downto*5,3), '当前价', price)
    # 调用新浪api
    content = requests.get('http://hq.sinajs.cn/list='+sinacode).text
    list = content.split(',')
    # print(content)
    # 第四列为当前价格
    # print(list[3])
    crt_price = float(list[3])
    max = float(list[4])
    min = float(list[5])


    mean = round((min+max)/2, 3)
    high = round(mean*(1+downto), 3)
    low = round(mean*(1-downto), 3)

    print("当前价", crt_price)
    print("今日中间价", mean)
    print("今日最高", max)
    print("今日最低", min)
    print("今日差价", round(max - min, 3))
    print("\n")
    print("成本价", price)
    print("数量", count)
    print("差价百分比", downto)
    print("\n")

    file.write("当前价"+ str(crt_price)+ "|| 今日中间价"+ str(mean)+" || 今日最高"+ str(max)+" || 今日最低"+str(min)+" || 今日差价 "+str(round(max - min, 3))+" \n|| 成本价 "+str(price)+" || 数量 "+ str(count)+" || 差价百分比 "+str(downto))
    file.write("\n")
    if(crt_price > price):
        cost = sell(price, count, high, round(count*sell_percent/100, 0)*100)
        cost = buy(cost[0], cost[1], price*(1-downto), cost[2])
        # money = sell(cost[0], cost[1], high)
        # cost = buy(money[0], money[1], low, money)
        # money = sell(cost[0], cost[1], high)
        # cost = buy(money[0], money[1], low, money)
        # money = sell(cost[0], cost[1], high)
        # cost = buy(money[0], money[1], low, money)
        # money = sell(cost[0], cost[1], high)
        # cost = buy(money[0], money[1], low, money)
        print("成本价降至", cost[0], "库存", cost[1]," 成本降低 "+ str((price-cost[0])/price))
        file.write("成本价降至 "+str(cost[0])+" 库存 "+ str(cost[1])+" 成本降低 "+ str((price-cost[0])/price))
        file.write("\n")
    else:

        cost = buy(price, count, low, moneybuy)
        cost = sell(cost[0], cost[1], round(price*(1+downto),3), round(cost[1]*sell_percent/100, 0)*100)
        cost = buy(cost[0], cost[1], round(price*(1-downto),3), cost[2])
        # cost = lower_buy(money[0], money[1], low, money)  322831
        # money = sell(cost[0], cost[1], high)
        # cost = lower_buy(money[0], money[1], low, money)
        # money = sell(cost[0], cost[1], high)
        # cost = lower_buy(money[0], money[1], low, money)
        # money = sell(cost[0], cost[1], high)
        # cost = lower_buy(money[0], money[1], low, money)
        print("成本价降至", cost[0],  "库存", cost[1]," 成本降低 "+ str((price-cost[0])/price))
        file.write("成本价降至 "+str(cost[0])+" 库存 "+ str(cost[1])+" 成本降低 "+ str((price-cost[0])/price))
        file.write("\n")


moneybuy = 20000
file = open("files/lower_price.txt", "a")
file.write("******************************************")
file.write(time.strftime("%Y%m%d", time.localtime(time.time())))
file.write("******************************************\n")
file.write("512760.SH" + "\n")
lower(2.522,131500,'512760.SH','sh512760',moneybuy,'fd')
file.write('159949.SZ'+"\n")
print("------------------------------------------------\n")
lower(1.049,129500,'159949.SZ','sz159949',moneybuy,'fd')
file.write('600377.SH'+"\n")
print("------------------------------------------------\n")
lower(10.003,2600,'600377.SH','sh600377',moneybuy,'e')
file.write("\n")

file.write("\n\n\n")
file.close()


当前价 2.51
今日中间价 2.512
今日最高 2.538
今日最低 2.486
今日差价 0.052


成本价 2.522
数量 131500
差价百分比 0.018536875495638382


买入价格： 2.465 买入数量： 8100.0 使用资金： 20000 单价变为： 2.519 降低： 0.003
卖出价格： 2.569 卖出数量： 125600.0 回收资金： 322666.4 资本增加： 322272.747 盈利： 6279.999999999977 单价变为： 2.099
买入价格： 2.475 买入数量： 130200.0 使用资金： 322272.747 单价变为： 2.439 降低： -0.34
成本价降至 2.439 库存 144200.0  成本降低 0.03291038858049157
------------------------------------------------

当前价 1.046
今日中间价 1.047
今日最高 1.057
今日最低 1.037
今日差价 0.02


成本价 1.049
数量 129500
差价百分比 0.018350810295519535


买入价格： 1.028 买入数量： 19500.0 使用资金： 20000 单价变为： 1.046 降低： 0.003
卖出价格： 1.068 卖出数量： 134100.0 回收资金： 143218.8 资本增加： 143044.073 盈利： 2950.2000000000025 单价变为： 0.86
买入价格： 1.03 买入数量： 138900.0 使用资金： 143044.073 单价变为： 1.014 降低： -0.154
成本价降至 1.014 库存 153800.0  成本降低 0.033365109628217274
------------------------------------------------

当前价 10.32
今日中间价 10.26
今日最高 10.35
今日最低 10.17
今日差价 0.18


成本价 10.003
数量 2600
差价百分比 0.006747975607317812


卖出价格： 10.329 卖出数量： 2300.0 回收资金： 23756.7 资本增加： 23